# Introduction

Welcome to the 2021 UCSF Biophysics Onboarding tutorial on protein modelling with pyRosetta! The contents of this tutorial were inspired by a pyRosetta tutorial produced on YouTube by Professor Sari Sabban of King Abdulaziz University.


Before getting started here, you should make sure that you have activated the pyRosetta conda environment and opened jupyter notebook with the following command:

`sudo jupyter notebook --allow-root`

Let's begin by importing pyRosetta into this Python session and initializing the module.

In [ ]:
# load some native Python modules
import os
# load pyRosetta module
from pyrosetta import *
from pyrosetta.toolbox import *
# initialize pyRosetta state (e.g. random number generator seed)
init()

### pyRosetta Poses from the RCSB and from PDB Files

pyRosetta is a useful tool for modelling and manipulating protein structures, so in order to make use of it we will need to load in a protein! For this exercise we will use the <i>de novo</i> designed protein Co-LOCKR from Lajoie et al. (2020), which exhibits switching behavior between two states. The authors have deposited the structure to the RCSB (the U.S.-based host of the Protein Data Bank) with the four-letter accession code 7JH5. We will load it using the pyRosetta function `pose_from_rcsb`.

In [ ]:
pose = pose_from_rcsb('7jh5')

You will notice that the above code returned quite a good deal of output.  This is because, in addition to downloading the structure from the RCSB in PDB format (creating the file `7JH5.pdb` in the working directory), it performed a series of operations to "clean" the PDB file, such as removing solvent and other non-protein atoms (creating the file `7JH5.clean.pdb`) and inferring the sidechain conformations of some residues which had missing sidechain information in the initial PDB file. This latter issue is why you can see mention of "packer tasks" and the "Dunbrack rotamer library." pyRosetta is scanning through a library of rotamers, or statistically-preferred sidechain conformers, to determine the set that leads to ideal packing of the protein fold in the regions where sidechains need to be inferred.

The appearance of these additional PDB files can be noted using `os.listdir` on the current working directory.

In [ ]:
os.listdir('.')

pyRosetta can also read in protein structures from PDB files directly, which can be useful when dealing with output from other software such as PyMOL and AlphaFold. An example of this follows:

In [ ]:
pose = pose_from_pdb('7JH5.clean.pdb')

### Pose from Sequence

pyRosetta also supports input of protein poses from sequence, although the pose object returned by this input function will not be initialized with atomic coordinates.  These will have to be inferred at a later stage.  Here we will load a pose from the sequence of a single "key" helix designed to pair with the Co-LOCKR protein.

In [ ]:
seq_key = 'SGGSDEARKAIARVKRESKRIVEDAERLIREAAAASEKISREAERLIRGG'
pose_key = pose_from_sequence(seq_key)

### Sequence from Pose

We know now how to get a pose from a sequence, but pyRosetta also lets us get a sequence from a pose. Let's get the sequence of that original pose we read in from the RCSB.

In [ ]:
pose.sequence()

### Residue Indexing

Let's now get some information about a particular residue in the protein, including its name (amino acid three-letter code) and its chain (indexed by a capital letter, 'A' or 'B' for 7JH5).

In [ ]:
print('total_residues :', pose.total_residue())

resn = 42 # feel free to change this value and execute the next three cells as many times as you'd like

assert resn >= 1 and resn <= pose.total_residue() # determine whether resn falls within its range
                                                  # note that pyRosetta indexes from 1 here, not from 0

res = pose.residue(resn).name()
chain = pose.pdb_info().chain(resn)

print('residue :', '\t', res)
print('chain :', '\t', chain)

pyRosetta assigns integer indices to each residue in the protein structure that ascend in unbroken succession from the N-terminus of the chain in which the residue can be found. These indices reset to 0 for each chain. These often do not agree with the residue indices in PDB files, which may account for residues that are not resolved in or have been truncated from the structure. To go between pyRosetta indices and Python indices, the following methods are used:

In [ ]:
idx = pose.pdb_info().number(resn)
py_idx = pose.pdb_info().pdb2pose(chain, idx)

print(idx, py_idx)

assert py_idx == resn # ensure that calculation of pynum has taken us full circle

### Residue Dihedral Angles

It is also easy to check the values (in degrees) of the phi, psi, and omega backbone dihedrals for a given residue. (Recall that omega, the dihedral within the peptide moiety, is almost always close to 180 degrees.)

In [ ]:
phi = pose.phi(resn)
psi = pose.psi(resn)
omega = pose.omega(resn)
print('phi :', phi)
print('psi :', psi)
print('omega :', omega)

### Number of Chains

We can check the number of chains in a pose object, as well as split one into a list of poses, one for each chain.

In [ ]:
num_chains = pose.num_chains()
print('number of chains :', num_chains)

poseA, poseB = pose.split_by_chain()

### Secondary Structure

It can be useful to think about the secondary structure of a protein (e.g. alpha helices and beta strands). pyRosetta has a simple built-in method for calculating secondary structure for each residue on the basis of its backbone phi and psi angles. In this scheme, 'H' denotes a helical residue, 'E' a residue in a beta strand or sheet, and 'L' a residue in an unstructured loop. It can be seen that the chains of 7JH5 each consist of a set of six looped helices.

In [ ]:
for chain_pose in [poseA, poseB]:
    chain_pose.display_secstruct()

# Viewing, Scoring, and Altering Poses

### Viewing Poses with PyMOL

PyMOL, like Chimera, is software for viewing the conformations of biomolecular systems such as proteins. The developers of pyRosetta have enabled pyRosetta and PyMOL to be linked together. After adding the correct lines to your .pymolrc file and starting PyMOL in another terminal window, the following will enable you to visualize your structures from a pyRosetta session:

In [ ]:
pymover = PyMOLMover()
pymover.apply(poseA)

### Scoring the Conformation

Central to Rosetta is its score function, which serves as a (very rough) approximation of the free energy (up to a constant) of a folded protein. The most commonly used (and default) score function is `ref2015`. This function can be loaded in as the "full-atom" score function and the weights of each term can be visualized by printing the score function object:

In [ ]:
scorefxn = get_fa_scorefxn()
print(scorefxn)

This score function behaves as a standard Python function, which takes a pose as input and returns a single float: the score of that pose.

In [ ]:
score = scorefxn(poseA)
print('score =', score)

We can also see a more detailed breakdown of the various contributions to the score.

In [ ]:
scorefxn.show(pose)

### Altering Dihedral Angles

pyRosetta is one of the only tools that allows you to alter the conformation of a protein one dihedral angle at a time, by swinging the remainder of the protein on the C-terminal side of a dihedral angle around the axis of the dihedral angle under consideration. An example follows:

In [ ]:
py_idx = pose.pdb_info().pdb2pose('A', 242) # residue 242 is the central residue of the last loop in 7JH5
phi = pose.phi(py_idx)

poseA.set_phi(py_idx, phi - 90)

new_score = scorefxn(poseA)
print('score =', new_score)

pymover.apply(poseA)

You will notice the score went up considerably after this dihedral was changed. This is to be expected, since a large amount of hydrophobic surface area was unburied when the helix swung upwards.

### Per-Residue Scores

It is possible to see the (unweighted) contributions of each term to a particular residue's score.

In [ ]:
poseA.energies().show(py_idx)

### Mutating Residues

pyRosetta makes it easy to mutate one residue to another. A subsequent round of repacking (that accompanies the action of mutation) helps to accommodate the new residue in a low-energy conformation.

In [ ]:
print('score =', scorefxn(poseA))
print()
print('sequence :', poseA.sequence()[py_idx-5:py_idx+5])
print()

mutate_residue(poseA, py_idx, 'Y')

print()
print('score =', scorefxn(poseA))
print()
print('sequence :', poseA.sequence()[py_idx-5:py_idx+5])

pymover.apply(poseA) # visualize residue 242 in PyMOL again; it is now a tyrosine

We notice that the energy contribution from this residue has become a bit more favorable with this mutation. This change is not enough, though, to offset the vast energy increase associated with rotating the alpha helix upward.

### Exporting a Structure

An altered structure can be exported to a PDB file, which makes pyRosetta quite useful as a tool for manipulating PDB files.

In [ ]:
poseA.dump_pdb('poseA.pdb')

# Relaxing a Pose

### The FastRelax Mover

Though the score has increased considerably, we will see if we can change the conformation of some of the sidechains in order to bring the score part of the way back down. In other words, we want to find a low score (free energy) configuration of the protein, subject to the backbone we have imposed upon it. This is done as follows, using a class called a "mover" to make changes to the structure (specifically we will utilize a mover that carries out the "FastRelax" protocol):

In [ ]:
relax = rosetta.protocols.relax.FastRelax()

Hypothetically we now would apply the mover to the pose in order to relax it, then see the improvement to the score that results. Since this takes some time, I have done this on my own device and included the result in the Github repository.

In [ ]:
# relax.apply(poseA)
# poseA.dump('results/poseA_relaxed.pdb')

poseA = pose_from_pdb('results/poseA_relaxed.pdb')

print()
print('score =', scorefxn(poseA))

pymover.apply(poseA)

We note that the score has considerably improved, and upon viewing the pose in PyMOL, we find that the hinge has again closed, assuming a pose similar to that in the RCSB.

# Protein Modelling with AlphaFold2

We will now turn to some protein modelling using AlphaFold2, a deep learning-based protein folding algorithm that offers the best-quality folding models available for most proteins. To do so, we will work with the ColabFold notebook, developed by Milot Mirdita, Sergey Ovchinnikov, and Martin Steinegger from the original Google Colab notebook released by DeepMind (the developers for AlphaFold2). We will begin by again printing out the sequence for Co-LOCKR:

In [ ]:
poseA.sequence()

We can fold this sequence at the standard ColabFold Notebook, found at:
https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb

We will also print out the sequence for another modelling target: Co-LOCKR without its C-terminal helix, but in the presence of the key helix. This will be a complex of two non-identical proteins, or a hetero-complex. Recall that we stored the integer index of the loop residue between the fifth and sixth helices as `py_idx`, and so we can acquire the sequence of the first five helices by slicing up to this value. We also stored the sequence of the key as `seq_key`. For hetero-complex modelling ColabFold takes as input the non-contiguous protein sequences separated by a colon. Let's make this sequence:

In [ ]:
seq_hetero = poseA.sequence()[:py_idx] + ':' + seq_key
print(seq_hetero)

We can fold this sequence at the advanced ColabFold Notebook, found at:
https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/beta/AlphaFold2_advanced.ipynb

# Working with C++ Rosetta

Hailey will now walk you through some of the function of Rosetta using the original C++ build of Rosetta on Wynton.